In [4]:

import pandas as pd 
import torch
import numpy as np 
import tqdm, pickle 
from echr_utils import  *

from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

from torch.utils.data import DataLoader
f = open('/net/scratch/jasonhu/legal_dec-sum/Dataset_ECHR/datasets/test.pickle', 'rb') 
test_dataset = pickle.load(f)
dt_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)
model = torch.load('/net/scratch/jasonhu/legal_dec-sum/Dataset_ECHR/saved_model/test2')
# optim = AdamW(model.parameters(), lr=5e-5)

if torch.cuda.is_available():
    device = torch.device('cuda') 
    print('用GPU！！！')
else: 
    device = torch.device('cpu')
    print('在用CPU。。。')
model.to(device)

softmax1 = torch.nn.Softmax(dim=1)


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:

for batch in tqdm.tqdm(dt_loader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    softmax1(outputs['logits'].to('cpu'))[:,1] # 这个可以是预测 violate 的一个分数，在 0 ~ 1 之间
